In [ ]:
import pandas as pd
import os
import numpy as np
import proplot as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import geopandas as gpd
import xarray as xr
import rioxarray as rioxr
from osgeo import gdal
os.getcwd()

#### 分流域统计分析

In [312]:
flist=[i for i in os.listdir('stat_data') if i[-4:]=='.csv']

In [317]:
#f=open('basin_gp.csv','w')
#f.write('group,lon,lat,nm\n')
#for nm in flist:
#    kh=nm.split('_')
#    f.write('%s,%s,%s,%s\n'%(kh[0],kh[1],kh[2][:-4],nm))
#f.close()

In [318]:
dff=pd.read_csv('basin_gp_all.csv')

In [322]:
bsnm=dff.basin.unique()
bsnm

array(['Yangtze', 'Brahmaputra', 'Yellow', 'Mekong', 'Salween',
       'Irrawaddy', 'Ganges'], dtype=object)

In [1224]:
for bn in bsnm:
    nnmm='Figs/%s.svg'%bn
    if os.path.exists(nnmm):
        continue
    print(bn)
    nmls=dff[dff.basin==bn]['nm'].values
    tmp_ls=[]
    for nm in nmls:
        df_tmp=pd.read_csv('stat_data/%s'%nm)
        tmp_ls.append(df_tmp)
    dff_all=pd.concat(tmp_ls,axis=0)
    break

Ganges


In [1225]:
dff_all.head()

,band,x,y,h,dh,slp,asp,lucc,asp_sign,slp_sign,h_sign
0,1,78.749243,31.500273,5022.0,26.172121,11.082644,128.08878,90.0,SE,10_20,5100.0
1,1,78.749243,31.499963,5018.0,24.312223,19.521154,156.44774,90.0,SE,10_20,5100.0
2,1,78.749243,31.499653,5004.0,10.891443,10.924911,103.73627,90.0,E,10_20,5100.0
3,1,78.749243,31.499343,5002.0,15.312435,14.826505,118.17859,90.0,SE,10_20,5100.0
4,1,78.749243,31.499033,4992.0,8.122230,18.275606,119.47589,90.0,SE,10_20,4900.0


In [1226]:
dff_all=dff_all[(dff_all['dh']>-30)&(dff_all['dh']<30)]

In [1227]:
###计算lucc重新分类
lcc=['grassland','shrubland','forest land','water','bare land','snow']
dff_all.loc[(dff_all.lucc==10)|(dff_all.lucc==30),'lucc_sign']=lcc[0]
dff_all.loc[(dff_all.lucc==40)|(dff_all.lucc==70),'lucc_sign']=lcc[1]
dff_all.loc[(dff_all.lucc==20),'lucc_sign']=lcc[2]
dff_all.loc[(dff_all.lucc==50)|(dff_all.lucc==60),'lucc_sign']=lcc[3]
dff_all.loc[(dff_all.lucc==80)|(dff_all.lucc==90),'lucc_sign']=lcc[4]
dff_all.loc[(dff_all.lucc==100),'lucc_sign']=lcc[5]

In [1228]:
#################################坡度分布
luc_st=dff_all.groupby(['slp_sign','lucc_sign'])['h'].count().to_xarray()

In [1229]:
dfpt_cnt=luc_st.to_dataframe().reset_index()
ddf_cnt=dfpt_cnt.pivot("lucc_sign", "slp_sign", "h").T
#换算为比例
my_cnt=ddf_cnt/(ddf_cnt.sum(axis=1).values).repeat(6).reshape(7,6)*100
pt_all=ddf_cnt.sum(axis=1)/ddf_cnt.sum(axis=1).sum()*100

In [1230]:
###计算dh情况
ds_slp_lcc=dff_all.groupby(['slp_sign','lucc_sign'])['dh'].mean().to_xarray()

In [1231]:
dfpt=ds_slp_lcc.to_dataframe().reset_index()
ddf=dfpt.pivot("lucc_sign", "slp_sign", "dh")

In [1232]:
ddf

slp_sign,0_10,10_20,20_30,30_40,40_50,50_60,>60
lucc_sign,,,,,,,
bare land,2.192020,1.559115,0.428734,-0.214477,-0.664122,-0.843741,-0.682660
forest land,3.151394,3.041150,1.603660,0.649871,-0.267390,-0.753229,-0.689036
grassland,1.776361,1.416927,0.340964,-0.508799,-1.079225,-1.316643,-1.033720
shrubland,1.102659,1.388305,0.486759,-0.059704,-0.390796,-0.728182,-1.291027
snow,-0.994349,-0.761560,-1.174226,-1.355202,-1.328055,-0.943303,-0.326314
water,-4.576384,-2.069369,0.099606,-0.582487,-3.168058,-4.839638,-4.910366


In [1233]:
#######################海拔分布
ExtractByStrip(dff_all,'h','h_sign500',500)
#dff_all['h_sign'].unique()

In [1234]:
def supp(dfpt_h):
    myh=range(950,6500,500)
    sss=[]
    for k in myh:
        jglen=len(dfpt_h[dfpt_h['h_sign500']==k])
        if jglen==0:
            ddf_new=pd.DataFrame([[k for i in range(len(lcc))],lcc,[np.nan for i in range(len(lcc))]],
                                 index=['h_sign500','lucc_sign',dfpt_h.columns[-1]]).T
            sss.append(ddf_new)    
    sss.append(dfpt_h)
    return pd.concat(sss)

In [1235]:
ds_h_lcc=dff_all.groupby(['h_sign500','lucc_sign'])['dh'].mean().to_xarray()
ds_h_lcc=ds_h_lcc.sel(h_sign500=slice(950,6450))
dfpt_h=ds_h_lcc.to_dataframe().reset_index()
dfpt_h=supp(dfpt_h)
ddf_h=dfpt_h.pivot("lucc_sign", "h_sign500", "dh")
ddf_h.columns=ddf_h.columns.astype(int).astype(str)

In [1236]:
luc_st_h=dff_all.groupby(['h_sign500','lucc_sign'])['h'].count().to_xarray()
luc_st_h=luc_st_h.sel(h_sign500=slice(950,6450))
dfpt_h_cnt=luc_st_h.to_dataframe().reset_index()

dfpt_h_cnt=supp(dfpt_h_cnt)
ddf_h_cnt=dfpt_h_cnt.pivot("lucc_sign", "h_sign500", "h").T
ddf_h_cnt.index=ddf_h_cnt.index.values.astype(int).astype(str)
#换算为比例
my_cnt_h=ddf_h_cnt/(ddf_h_cnt.sum(axis=1).values).repeat(6).reshape(12,6)*100
pt_all_h=ddf_h_cnt.sum(axis=1)/ddf_h_cnt.sum(axis=1).sum()*100

In [1237]:
dfpt_h_cnt['h_sign500'].unique()

array([ 950., 1450., 1950., 2450., 2950., 3450., 3950., 4450., 4950.,
       5450., 5950., 6450.])

In [1238]:
#########################坡向分布
ds_asp_lcc=dff_all.groupby(['asp_sign','lucc_sign'])['dh'].mean().to_xarray()
dfpt_asp=ds_asp_lcc.to_dataframe().reset_index()
ddf_asp=dfpt_asp.pivot("lucc_sign", "asp_sign", "dh")

In [1239]:
luc_st_asp=dff_all.groupby(['asp_sign','lucc_sign'])['h'].count().to_xarray()
dfpt_asp_cnt=luc_st_asp.to_dataframe().reset_index()
ddf_asp_cnt=dfpt_asp_cnt.pivot("lucc_sign", "asp_sign", "h").T
#换算为比例
my_cnt_asp=ddf_asp_cnt/(ddf_asp_cnt.sum(axis=1).values).repeat(6).reshape(8,6)*100
pt_all_asp=ddf_asp_cnt.sum(axis=1)/ddf_asp_cnt.sum(axis=1).sum()*100

In [1240]:
pt_all_h.index

Index(['950', '1450', '1950', '2450', '2950', '3450', '3950', '4450', '4950',
       '5450', '5950', '6450'],
      dtype='object')

In [1241]:
ddf

slp_sign,0_10,10_20,20_30,30_40,40_50,50_60,>60
lucc_sign,,,,,,,
bare land,2.192020,1.559115,0.428734,-0.214477,-0.664122,-0.843741,-0.682660
forest land,3.151394,3.041150,1.603660,0.649871,-0.267390,-0.753229,-0.689036
grassland,1.776361,1.416927,0.340964,-0.508799,-1.079225,-1.316643,-1.033720
shrubland,1.102659,1.388305,0.486759,-0.059704,-0.390796,-0.728182,-1.291027
snow,-0.994349,-0.761560,-1.174226,-1.355202,-1.328055,-0.943303,-0.326314
water,-4.576384,-2.069369,0.099606,-0.582487,-3.168058,-4.839638,-4.910366


In [ ]:
plt.rcParams['font.size'] = 8
plt.rcParams['font.sans-serif']

arr=[[1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,5,5,5,5,5,5,5,5],
     [1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,5,5,5,5,5,5,5,5],
     [1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,5,5,5,5,5,5,5,5],
     [1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,5,5,5,5,5,5,5,5],
     [1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,5,5,5,5,5,5,5,5],
     [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2],
     [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2],
     [4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4],
     [4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4],
     [4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4],
     [6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6],
     [6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6]]
fig,ax=plot.subplots(arr,figsize=('18cm','15cm'),sharex=0,sharey=0,hspace=0.2,wspace=0.05)

ax[0].heatmap(ddf,vmin=-20,vmax=20,cmap='PiYG',labels_kw={'fontsize':6})#,labels=True
val_h=ddf.values
x,y=val_h.shape
for i in range(y):
    for j in range(x): 
        if np.isnan(val_h[j][i]):
            pass
        else:
            ax[0].text(i-0.4,j-0.2,str(round(val_h[j][i],2)),fontsize=6)
ax[0].format(xlabel='Slope range (°)',ylabel='',xrotation=30)

lg=ax[1].barh(my_cnt, cycle=['y','g','#94DDBC','#94B7DD','#EDFDFC','b'], edgecolor='k', stacked=True)
ax[1].legend(bbox_to_anchor=(0, 1),loc='lower left',ncol=10)
ax[1].format(xlim=(0,123),xticks=range(0,110,10),ylabel='',xlabel='')

ax1t=ax[1].twiny()
ax1t.plot(pt_all.values,pt_all.index,lw=1,marker='_',c='blue7',markersize=10)
ax1t.fill_between([0,40],-1,10,facecolor='gray',alpha=0.5)
ax1t.format(ylim=(-0.50,6.5),xlim=(-210,40),xticks=[0,10,20,30,40],xcolor='blue7',ylabel='')
####################################################################
#ax[2].heatmap(ddf_h.fillna(0),vmin=-10,vmax=10,color='',labels=True)
###ddf_h[ddf_h<20]  for ganges
ax[2].heatmap(ddf_h,vmin=-10,vmax=10,cmap='PiYG')
val_h=ddf_h.values
x,y=val_h.shape
for i in range(y):
    for j in range(x): 
        if np.isnan(val_h[j][i]):
            pass
        else:
            ax[2].text(i-0.4,j-0.2,str(round(val_h[j][i],2)),fontsize=6)
ax[2].format(yticklabels=[],ylabel='',xlabel='Elevation range (m)',xrotation=30)

lg=ax[3].barh(my_cnt_h, cycle=['y','g','#94DDBC','#94B7DD','#EDFDFC','b'], edgecolor='k', stacked=True)
ax[3].format(xlim=(0,123),xticks=range(0,110,10),ylabel='',xlabel='')

ax2t=ax[3].twiny()
ax2t.plot(pt_all_h.values,pt_all_h.index,lw=1,marker='_',c='blue7',markersize=10)
ax2t.fill_between([0,40],-1,12,facecolor='gray',alpha=0.5)
ax2t.format(ylim=(-0.50,11.5),xlim=(-210,40),xticks=[0,10,20,30,40],xcolor='blue7',ylabel='')
########################################################################
ax[4].heatmap(ddf_asp,vmin=-10,vmax=10,cmap='PiYG',labels=True,labels_kw={'fontsize':6})
ax[4].format(yticklabels=[],ylabel='',xlabel='Aspect')

lg=ax[5].barh(my_cnt_asp, cycle=['y','g','#94DDBC','#94B7DD','#EDFDFC','b'], edgecolor='k', stacked=True)
ax[5].format(xlim=(0,123),xticks=range(0,110,10),ylabel='',xlabel='')

ax4t=ax[5].twiny()
ax4t.plot(pt_all_asp.values,pt_all_asp.index,lw=1,marker='_',c='blue7',markersize=10)
ax4t.fill_between([0,40],-1,9,facecolor='gray',alpha=0.5)
ax4t.format(ylim=(-0.50,7.5),xlim=(-210,40),xticks=[0,10,20,30,40],xcolor='blue7',ylabel='')
plt.savefig('Figs/%s.svg'%bn,dpi=300,bbox_inches='tight')

In [1243]:
##最后一次运行了全流域的数据，存储一下，方便下次用
#dff_all.to_csv('stat_data_all.csv')